In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("/kaggle/input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv")

In [ ]:
df.info()

In [ ]:
df.head(5)

In [ ]:
df.isna().sum()

In [ ]:
#replacing null values with 0
df['salary'].fillna(0,inplace=True)
df['salary'].isna().any()

In [ ]:
df['gender']=df['gender'].map({'M':0,'F':1})

In [ ]:
df['hsc_b']=df['hsc_b'].map({'Central':1,'Others':0})

In [ ]:
df['ssc_b']=df['ssc_b'].map({'Central':1,'Others':0})

In [ ]:
df_subjects=pd.get_dummies(df['hsc_s'])
df=df.merge(df_subjects,on=df.index)

In [ ]:
df.drop('key_0',axis=1,inplace=True)
df.drop('hsc_s',axis=1,inplace=True)

In [ ]:
df_deg=pd.get_dummies(df['degree_t'])
df=df.merge(df_deg,on=df.index)
df.drop('key_0',axis=1,inplace=True)
df.drop('degree_t',axis=1,inplace=True)

In [ ]:
df['status']=df['status'].map({'Placed':1,'Not Placed':0})

In [ ]:
df['specialisation']=df['specialisation'].map({'Mkt&Fin':1,'Mkt&HR':0})

In [ ]:
df['workex']=df['workex'].map({'Yes':1,'No':0})

In [ ]:
df['specialisation']=df['specialisation'].map({'Mkt&Fin':1,'Mkt&HR':0})

In [ ]:
df.head()

In [ ]:
#gender wise dictribution over placed vs non placed
sns.catplot('gender',data=df,kind='count',hue='status',palette='rocket')

In [ ]:
#ssc score distribution
sns.distplot(df['ssc_p'], kde=False)
plt.title('Distribution of SSC Percentage')
plt.xlabel('SSC %')

In [ ]:
sns.catplot(y='ssc_p', x='status', data=df)
plt.xlabel('Employment Status vs ssc_p')
plt.ylabel('SSC %')

In [ ]:
#status vs ssc_b(board of ssc)
sns.countplot(x='ssc_b', hue='status', data=df)

In [ ]:
#status vs hsc_b(board of hsc)
sns.countplot(x='hsc_b', hue='status', data=df)

In [ ]:
sns.catplot(y='mba_p', x='status', data=df)
plt.xlabel('Employment Status')
plt.ylabel('MBA %')

In [ ]:
#dropping categorical variables
df.drop(['ssc_b','hsc_b', 'Arts','Commerce','Science','Comm&Mgmt','Others','Sci&Tech', 'salary','specialisation'], axis=1, inplace=True)

In [ ]:
# Seperating Features and Target
y = df['status']
X = df.drop('status', axis=1)

In [ ]:
df.isna().sum()

In [ ]:
#Train Test Split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3,random_state=0)
X_train.shape, Y_train.shape, X_test.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
reg_log=LogisticRegression()
reg_log.fit(X_train,Y_train)

In [ ]:
reg_log.score(X_train,Y_train)

In [ ]:
y_pred_log=reg_log.predict(X_test)
reg_log.score(X_test,Y_test)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
print("RMSE: ", np.sqrt(mean_squared_error(Y_test, y_pred_log)))

print("R2: ", r2_score(Y_test, y_pred_log))

In [ ]:
conf_mat_log=confusion_matrix(y_pred_log,Y_test)
labels=['Unplaced','Placed']
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111)
cax = sns.heatmap(conf_mat_log,ax=ax,annot=True,cmap='viridis')
ax.xaxis.set_ticklabels(['Unplaced', 'Placed'])
ax.yaxis.set_ticklabels(['Unplaced', 'Placed'],rotation=0)
ax.set_xlabel('Predicted',size=15)
ax.set_ylabel('Actual',size=15)
plt.title('Confusion matrix Logistic Regression',size=15)
plt.figure(figsize=(10,8))